In [96]:
#-------------------------------------------------------IMPORT----------------------------------------------------------------------------#
#dieu
import numpy as np
import pandas as pd
import string
from collections import defaultdict
from time import time
# Sklearn
from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer , TfidfTransformer
from sklearn.model_selection import ShuffleSplit
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import recall_score , precision_score ,f1_score
from sklearn import metrics , svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier, LogisticRegression, LogisticRegressionCV
from sklearn.naive_bayes import MultinomialNB, CategoricalNB, ComplementNB, BernoulliNB
from sklearn.svm import SVC, LinearSVC
#vis
import plotly.express as px
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import plotly as py
from plotly.offline import init_notebook_mode, iplot, plot
#NLP
import nltk
from nltk.corpus import stopwords , wordnet as wn
from nltk import wordpunct_tokenize , WordNetLemmatizer ,sent_tokenize ,  word_tokenize
from nltk.stem import PorterStemmer , LancasterStemmer
from nltk.stem.snowball import SnowballStemmer  

In [42]:
#------------------------------------------------------DATAFRAME DISPLAY------------------------------------------------------------------#
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', -1)
pd.reset_option("^display") ##Reset display option

In [65]:
#-----------------------------------------------------------FUNCTIONS--------------------------------------------------------------------#
def get_top_n_words(corpus,d,n=None):
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in     vec.vocabulary_.items()]
    if d == "up" :
        words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
        return words_freq[:n]
    elif d == "down" :
        words_freq=sorted(words_freq, key = lambda x: x[1], reverse=False)
        return words_freq[:n]

def tokenize(corpus):
    #text = ''.join([ch for ch in text if ch not in dataEF["Text"]])
    tokens = nltk.word_tokenize()
    lemma = WordNetLemmatizer()
    stemmer = SnowballStemmer("english")
    # tokens = [lemmatizer.lemmatize(token) for token in tokens] 
    #OR
    # tokens = [lemma.lemmatize(lemma.lemmatize(lemma.lemmatize(w,'v'),'n'),'a')for w in tokens]
    return [stemmer.stem(token) for token in tokens]

def run_pipes(pipes, splits=10, test_size=0.2, seed=42):  
    res = defaultdict(list)
    spliter = ShuffleSplit(n_splits=splits, test_size=test_size, random_state=seed)
    for idx_train, idx_test in spliter.split(corpus):
        for pipe in pipes:
            # name of the model
            name = "-".join([x[0] for x in pipe.steps])
            
            # extract datasets
            X_train = corpus[idx_train]
            X_test = corpus[idx_test]
            y_train = targets[idx_train]
            y_test = targets[idx_test]
            
            # Learn
            start = time()
            pipe.fit(X_train, y_train)
            fit_time = time() - start
            
            # predict and save results
            y = pipe.predict(X_test)
            res[name].append([
                fit_time,
                f1_score(y_test, y, average = 'micro')
            ])
    return res

def print_table(res):
    # Compute mean and std
    final = {}
    for model in res:
        arr = np.array(res[model])
        final[model] = {
            "time" : arr[:, 0].mean().round(2),
            "f1_score": [arr[:,1].mean().round(5),arr[:,1].std().round(5)]
                    }

    df = pd.DataFrame.from_dict(final, orient="index").round(3)
    return df

In [44]:
# df0_pre = pd.read_csv('../data/emotion_final.csv')

# exclude = set(string.punctuation) # exclude = punctuation strings
# stop_word = stopwords.words('english') # we choosing stop words of english dict
# stop_word_punct = stop_word.extend(exclude) # we add strings punctions to stop word dict

# df0_pre.tokenized_sents = df0_pre.apply(lambda row: word_tokenize(row['Text']), axis=1) # Tokenization
# df0_pre.tokenized_sents = df0_pre.tokenized_sents.apply(lambda x: [item for item in x if item not in stop_word_punct])
# df0_pre.tokenized_sents = [[lemma.lemmatize(word) for word in each if word not in stop_sw] for each in df0_pre.tokenized_sents] 
# df0_pre.tokenized_sents = df0_pre.tokenized_sents.apply(lambda x: [stemmer.stem(y) for y in x]) # Stem every word.
# df0_pre.tokenized_sents = df0_pre.tokenized_sents.apply(lambda x: [porter.stem(y) for y in x]) # Stem every word.
# df0_pre.tokenized_sents = df0_pre.tokenized_sents.apply(lambda x: [lancaster.stem(y) for y in x]) # Stem every word.
# #delete list in list, **********need to be in one line*********
# dz = df0_pre.tokenized_sents
# dz = [[' '.join(i)][0] for i in dz]
# df0_pre.tokenized_sents = dz

In [45]:
df1 = pd.read_csv('../data/emotion_final.csv')

exclude = set(string.punctuation) # exclude = punctuation strings
stop_word = stopwords.words('english') # we choosing stop words of english dict
stop_word_punct = stop_sw.extend(exclude) # we add strings punctions to stop word dict

corpus = np.array(df1["Text"])
targets = np.array(df1["Emotion"])
targets = np.array([1 if x == "sadness" else 2 if x=="anger" else 3 if x=="love" else 4 if x=="surprise" else 5 if x=="fear" else 6 for x in targets])

NameError: name 'stop_sw' is not defined

In [46]:
pipe0 = Pipeline([
    ('vect', CountVectorizer()),
    ('sgd', SGDClassifier()),
])
pipe1 = Pipeline([
    ('vect', CountVectorizer()),
    ('svm', SVC()),
])
pipe2 = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('sgd', SGDClassifier()),
])
pipe3 = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('svm', LinearSVC()),
])
pipe4 = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('svm', SVC()),
])
pipe5 = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('logit', LogisticRegression()),
])
pipe6 = Pipeline([
    ('vect', CountVectorizer()),
    ('mult_nb', MultinomialNB()),
])
pipe7 = Pipeline([
    ('vect', CountVectorizer()),
    ('compl_nb', ComplementNB()),
])
pipe8 = Pipeline([
    ('vect', CountVectorizer()),
    ('bern_nb', BernoulliNB()),
])

In [47]:
# res = run_pipes([pipe0, pipe1, pipe2, pipe3, pipe4, pipe5, pipe6, pipe7, pipe8], splits=5)
# print_table(res)

In [48]:
pipe0 = Pipeline([
    ('vect', CountVectorizer()),
    ('sgd', SGDClassifier()),
])
pipe1 = Pipeline([
    ('vect', CountVectorizer()),
    ('svm', SVC()),
])
pipe2 = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('sgd', SGDClassifier()),
])
pipe3 = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('svm', LinearSVC()),
])
pipe4 = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('svm', SVC()),
])
pipe5 = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('logit', LogisticRegression()),
])
pipe6 = Pipeline([
    ('vect', CountVectorizer()),
    ('mult_nb', MultinomialNB()),
])
pipe7 = Pipeline([
    ('vect', CountVectorizer()),
    ('compl_nb', ComplementNB()),
])
pipe8 = Pipeline([
    ('vect', CountVectorizer()),
    ('bern_nb', BernoulliNB()),
])

In [49]:
# run base pipes
res = run_pipes([pipe0, pipe1, pipe2, pipe3, pipe4, pipe5, pipe6, pipe7, pipe8], splits=5)
print_table(res)

# 	time	f1_score
# vect-sgd	0.64	[0.887, 0.005]
# vect-svm	91.26	[0.787, 0.003]
# vect-tfidf-sgd	0.50	[0.89, 0.003]
# vect-tfidf-svml	0.59	[0.891, 0.007]
# vect-tfidf-svm	97.59	[0.845, 0.006]
# vect-tfidf-logit	4.45	[0.854, 0.006]
# vect-mult_nb	0.25	[0.745, 0.005]
# vect-compl_nb	0.24	[0.877, 0.003]
# vect-bern_nb	0.25	[0.655, 0.008]

NameError: name 'corpus' is not defined

In [50]:
df1 = pd.read_csv('../data/emotion_final.csv')

# print(df1['Emotion'].unique())
df_count_emotion = df1.groupby(['Emotion']).size().reset_index(name='Count')
print(df_count_emotion) 

    Emotion  Count
0     anger   2993
1      fear   2652
2     happy   7029
3      love   1641
4   sadness   6265
5  surprise    879


In [51]:
exclude = set(string.punctuation) # exclude = punctuation strings
stop_words = stopwords.words('english') # we choosing stop words of english dict
stop_words.extend(exclude) # we add strings punctions to stop word dict
lemma = WordNetLemmatizer()
stemmer = SnowballStemmer("english") # we choosing the language english for the stemmization 
porter = PorterStemmer() 
lancaster=LancasterStemmer()


In [52]:
df_pre = pd.read_csv('../data/emotion_final.csv')
df_pr_pre['Text'] = df_pre.apply(lambda row: word_tokenize(row['Text']), axis=1) # Tokenization
df_pre['Text'] = df_pre['Text'].apply(lambda x: [item for item in x if item not in stop_words]) # Stop wordization :) coucou anne-laure
df_pre['Text'] = [[lemma.lemmatize(word) for word in each if word not in stop_sw] for each in df_pre['Text']]  # Lemmization
# df_pre['Text'] = df1['Text'].apply(lambda x: [stemmer.stem(y) for y in x]) # Stem every word. with snowball('english')
# df_pre['Text'] = df1['Text'].apply(lambda x: [porter.stem(y) for y in x]) # Stem every word. with porter
# df_pre['Text'] = df1['Text'].apply(lambda x: [lancaster.stem(y) for y in x]) # Stem every word. with lancaster
dz = df_pre['Text']
dz = [[' '.join(i)][0] for i in dz] 
df_pre['Text'] = dz
df_pre

NameError: name 'df_pr_pre' is not defined

In [ ]:
df1 = pd.read_csv('../data/emotion_final.csv')
df1['Text'] = df1.apply(lambda row: word_tokenize(row['Text']), axis=1) # Tokenization
df1['Text'] = df1['Text'].apply(lambda x: [item for item in x if item not in stop_sw]) # Stop wordization :) coucou anne-laure
# df1['Text'] = [[lemma.lemmatize(word) for word in each if word not in stop_sw] for each in df1['Text']]  # Lemmization
# df1['Text'] = df1['Text'].apply(lambda x: [stemmer.stem(y) for y in x]) # Stem every word. with snowball('english')
df1['Text'] = df1['Text'].apply(lambda x: [porter.stem(y) for y in x]) # Stem every word. with porter
df1['Text'] = df1['Text'].apply(lambda x: [lancaster.stem(y) for y in x]) # Stem every word. with lancaster

dz = df1['Text']
dz = [[' '.join(i)][0] for i in dz] 

df1['Text'] = dz

df1

In [ ]:
data = {'Emotion':df1['Emotion'],'Text':df1['Text']}
df28 = pd.DataFrame(data=data)
df28

In [55]:
df1 = pd.read_csv('../data/emotion_final.csv')
df1['Text'] = df1.apply(lambda row: word_tokenize(row['Text']), axis=1) # Tokenization
df1['Text'] = df1['Text'].apply(lambda x: [item for item in x if item not in stop_sw]) # Stop wordization :) coucou anne-laure
# df1['Text'] = [[lemma.lemmatize(word) for word in each if word not in stop_sw] for each in df1['Text']]  # Lemmization
df1['Text'] = df1['Text'].apply(lambda x: [stemmer.stem(y) for y in x]) # Stem every word. with snowball('english')
df1['Text'] = df1['Text'].apply(lambda x: [porter.stem(y) for y in x]) # Stem every word. with porter
df1['Text'] = df1['Text'].apply(lambda x: [lancaster.stem(y) for y in x]) # Stem every word. with lancaster

dz = df1['Text']
dz = [[' '.join(i)][0] for i in dz] 

df1['Text'] = dz

df1

NameError: name 'stop_sw' is not defined

In [ ]:
df2 = pd.read_csv('../data/text_emotion.csv')

df2['tokenized_sents'] = df2.apply(lambda row: word_tokenize(row['content']), axis=1) # Tokenization
df2['tokenized_sents'] = df2['tokenized_sents'].apply(lambda x: [item for item in x if item not in stop_sw])
df2.tokenized_sents = [[lemma.lemmatize(word) for word in each if word not in stop_sw] for each in df2.tokenized_sents] 
df2['tokenized_sents'] = df2['tokenized_sents'].apply(lambda x: [stemmer.stem(y) for y in x]) # Stem every word.
df2['tokenized_sents'] = df2['tokenized_sents'].apply(lambda x: [porter.stem(y) for y in x]) # Stem every word.
df2['tokenized_sents'] = df2['tokenized_sents'].apply(lambda x: [lancaster.stem(y) for y in x]) # Stem every word.

df2

In [ ]:
corpus = df2['tokenized_sents']
x = corpus
x = [[' '.join(i)][0] for i in x]

df2['tokenized_sents'] = x
df2

In [58]:
corpus = np.array(df1['Text'])
x = corpus

# TF-IDF Vectorizer 
v_tf = TfidfVectorizer(stop_words=stop_sw, tokenizer=tokenize, ngram_range=(1,2))
x_tf = v_tf.fit_transform(x)
# TF-IDF Model
idf_values = dict(zip(v_tf.get_feature_names(), v_tf.idf_))
words_name = v_tf.get_feature_names()

print("-- TF-IDF vector shape :",x_tf.shape)

NameError: name 'stop_sw' is not defined

In [ ]:
corpus = np.array(df1['Text'])
x = corpus

cv = CountVectorizer(stop_words=stop_sw, tokenizer=tokenize, ngram_range=(1,2))
x_cv = cv.fit_transform(x) ## Count Vectorizer Model
print("-- Count_Vectorizer shape :",x_cv.shape)

In [ ]:
# Train pack with TF-IDF Vectorizer 
x_tf_train, x_tf_test, y_train, y_test = train_test_split(x_tf, y, test_size=0.20, random_state=0)
print('-- Training pack TF-IDF:',"\n")
print('-- x_tf_train rows :',np.size(x_tf_train))
print('-- x_tf_test rows :',np.size(x_tf_test))
print('-- ratio in % :', round(np.size(x_tf_test)/(np.size(x_tf_train)+np.size(x_tf_test))*100,2),"\n") 


In [ ]:
corpus = np.array(df2['content'])
x = corpus

# TF-IDF Vectorizer 
v_tf = TfidfVectorizer(stop_words=stop_sw, tokenizer=tokenize, ngram_range=(1,2))
x_tf = v_tf.fit_transform(x)
# TF-IDF Model
idf_values = dict(zip(v_tf.get_feature_names(), v_tf.idf_))
words_name = v_tf.get_feature_names()

print("-- TF-IDF vector shape :",x_tf.shape)

In [ ]:
corpus = np.array(df2['content'])
x = corpus

cv = CountVectorizer(stop_words=stop_sw, tokenizer=tokenize, ngram_range=(1,2))
x_cv = cv.fit_transform(x) ## Count Vectorizer Model
print("-- Count_Vectorizer shape :",x_cv.shape)

In [66]:
df1 = pd.read_csv('../data/emotion_final.csv')

x = df1.Text
y = df1.Emotion

freq_top = get_top_n_words(x,"up",100)


df_up = pd.DataFrame(freq_top, columns =['Word','Number of times'])
y_nbr = df_up['Number of times']
x_word = df_up['Word']
# df_down = pd.DataFrame(freq_down, columns =['Word','Number of times'])


In [67]:


# freq = subsample(wrank)
# r = np.arange(len(freq))

trace = go.Bar(
                x = x_word.head(30),
                y = y_nbr,
                name = "Le score universitaire pour le transfert de connaissances par pays",
                marker = dict(color = 'rgba(255, 87, 51, 0.5)', line = dict(color ='rgb(0,0,0)',width =2.5)),
                text = df_up['Word'])

layout = go.Layout(barmode = "group",
                  title = 'Fréquence d’apparition des mots ',
                  yaxis = dict(title = 'word frequency'),
                  xaxis = dict(title = 'word rank'))
fig = go.Figure(data = trace, layout = layout)
iplot(fig)

In [94]:

dftableau = df1
ParGroup = dftableau.groupby('Emotion')
y1=ParGroup['Emotion'].size().sort_values(ascending=False)


trace = go.Bar(
            x = y1.index.get_level_values(0).tolist(),                  
            y = y1,
            name = 'Income',
            type = 'bar',
                )

layout = go.Layout(
            xaxis = {'title': 'Emotions'},
            yaxis = {'title': 'Occurence'},
            barmode = 'relative',
            title = 'Emotions ranked by occurence in Emotion_final')

fig = go.Figure(data = trace, layout = layout)

iplot(fig)

In [112]:
df1 = pd.read_csv('../data/emotion_final.csv')

x = df1.groupby('Emotion')

trace = go.Bar(
            x = x,                  
            y = x,
            name = 'Income',
            type = 'bar',
                )

layout = go.Layout(
            xaxis = {'title': 'Emotions'},
            yaxis = {'title': 'Occurence'},
            barmode = 'relative',
            title = 'Emotions ranked by occurence in Emotion_final')

fig = go.Figure(data = trace, layout = layout)

iplot(fig)

ValueError: 
    Invalid value of type 'pandas.core.groupby.generic.DataFrameGroupBy' received for the 'x' property of bar
        Received value: <pandas.core.groupby.generic.DataFrameGroupBy object at 0x7f155f419c90>

    The 'x' property is an array that may be specified as a tuple,
    list, numpy array, or pandas Series

In [132]:

df1 = pd.read_csv('../data/emotion_final.csv')


emotion_hist = go.Figure(px.histogram(df1, x="Emotion", color= "Emotion", title="Histogramme Emotion").update_xaxes(categoryorder ="total descending"))
emotion_hist.update_layout(
    paper_bgcolor='rgba(0,0,0,0.65)',
    plot_bgcolor='rgba(0,0,0,0.65)')


fig.show()

In [130]:

df1 = pd.read_csv('../data/emotion_final.csv')

x = df1['Emotion']

data = go.Histogram(
    x="Emotion"
    
)

layout = go.Layout()

fig = go.Figure(data = trace, layout = layout)

ValueError: 
    Invalid value of type 'builtins.str' received for the 'x' property of histogram
        Received value: 'Emotion'

    The 'x' property is an array that may be specified as a tuple,
    list, numpy array, or pandas Series